# Preliminaries

All of the same preliminaries

In [1]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt

def norm_vec(v):
    mag = np.linalg.norm(v)
    if mag == 0:
        return v
    return v / np.linalg.norm(v)

from sklearn.preprocessing import normalize

def normalize_rows(x):
    return normalize(x, axis=1)

def normalize_columns(x):
    return normalize(x, axis=0)

def check_float(potential_float):
    try:
        float(potential_float)
        return True
    except ValueError:
        return False

def round_if_float(v, prec=3):
    if check_float(v):
        return round(float(v), prec)
    return v

from IPython.core.display import display, HTML
def list_table(the_list, color_nums=False):
    html = ["<table style= 'border: 1px solid black; display:inline-block'>"]
    for row in the_list:
        html.append("<tr>")
        for col in row:
            if color_nums and check_float(col) and not float(col) == 0:
                html.append("<td align='left' style='border: .5px solid gray; color: {1}; font-weight: bold'>{0}</td>".format(round_if_float(col), color_nums))
            else:
                html.append("<td align='left' style='border: .5px solid gray;'>{0}</td>".format(round_if_float(col)))
        html.append("</tr>")
    html.append("</table>")
    return display(HTML(''.join(html)))

def show_labeled_table(mat, col_names=None, row_names=None, nrows=10, ncols=10, color_nums="red"):
    sml = mat[:nrows, :ncols]
    if col_names is not None:
        sml = np.vstack([col_names[:ncols], sml])
    if row_names is not None:
        rnames = [[p] for p in row_names[:nrows]]
        if col_names is not None:
            new_col = np.array([["_"]] + rnames)
        else:
            new_col = np.array(rnames)
        sml = np.hstack((new_col, sml))
    return list_table(sml, color_nums)

def compute_doc_vector(tdoc, vocab):
    return np.array([tdoc.count(w) for w in vocab])

# Word vectors for the seasons

## Load the training corpus. 

This time we'll split it into sentences

In [2]:
import re
fname = 'corpora/seasons_training.txt'
f = open(fname)
raw = f.read().lower()
whole_training_docs = re.findall(r"<text>([\s\S]*?)</text>", raw)

In [3]:
training_docs = []
para_names = []
for i, d in enumerate(whole_training_docs):
    new_docs = nltk.sent_tokenize(d)
    training_docs += new_docs
    new_names = ["d{}p{}".format(i, p) for p in range(len(new_docs))]
    para_names += new_names

Now we have close to 14000 documents

In [4]:
len(training_docs)

13917

## Tokenize the training documents

Now we tokenize the ~14000 training documents

In [5]:
from seasons_module import seasons_tokenize
tokenized_training_docs = []
for doc in training_docs:
    tdoc = seasons_tokenize(doc)
    tokenized_training_docs.append(tdoc)

In [6]:
fdist = nltk.FreqDist()
for doc in tokenized_training_docs:
    fdist.update(doc)

f = open("lists/seasons_stop_list.txt")
stop_list = set(f.read().split("\n"))

full_vocab = [w[0] for w in fdist.most_common() if w[0] not in stop_list]
vocab = full_vocab[:500]

## Get the document vectors


When I first tried to compute document vectors executing it took a long time to run - 5 or 10 minutes. I had to poke around the internet and fiddle a bit. I figured out that building the big matrix in chunks and then putting those chunks together made a big difference.

In [7]:
from IPython.display import display, clear_output

def wfactor(tf):
    if tf == 0:
        result = 0
    else:
        result = (1 + np.log(tf))
    return result

def compute_doc_vector(token_list, vocab):
    return np.array([wfactor(token_list.count(word)) for word in vocab])

chunk_list = []
empty_rows = 0
chunk = np.array([], dtype=np.int64).reshape(0,len(vocab))
for i, tdoc in enumerate(tokenized_training_docs):
    new_row = compute_doc_vector(tdoc, vocab)
    if i % 500 == 0:
        clear_output(wait=True)
        display(str(i))
        chunk_list.append(chunk)
        chunk = np.array([], dtype=np.int64).reshape(0,len(vocab))
    if np.linalg.norm(new_row) == 0:
        empty_rows += 1
        continue
    chunk = np.concatenate([chunk, np.array([new_row])])
    
chunk_list.append(chunk)
clear_output(wait=True)
display(str(i) + " joining chunks")

training_dt_matrix2 = np.concatenate(chunk_list)
print("empty rows {}".format(empty_rows))

'13916 joining chunks'

empty rows 670


In [10]:
training_dt_matrix2.shape

(13247, 500)

In [11]:
show_labeled_table(training_dt_matrix2, vocab, para_names, nrows=15, ncols=10)

_,solar,climate,surface,energy,temperature,atmosphere,year,air,hemisphere,time
d0p0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d0p1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d0p2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d0p3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
d0p4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
d0p5,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
d0p6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
d0p7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d0p8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d0p9,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Word vectors with reduced dimensions

If, as before, we think of each of the columns in our table as a vector for the words, then each word is a vector with 13,264 dimensions.

We can use something called the singular value decomposition to reduce the number of dimensions. You read about this in the paper by Landauer. This is the trick that is used in latent semantic analysis.

First we pick a number of dimensions. I'm using a slider widget simply because it's fun.

In [12]:
import ipywidgets as widgets
w = widgets.IntSlider(value=100, max=200, description="rdims")
display(w)

IntSlider(value=100, description='rdims', max=200)

In [13]:
rdims = w.value
# do the SVD and reduce dimensions
T, S, Dt = np.linalg.svd(training_dt_matrix2.transpose(), full_matrices = False)
T_reduced = T[:, 0:rdims]
T_normed = normalize_rows(T_reduced)
print("shape of t_normed is {}".format(T_normed.shape))
show_labeled_table(T_normed, None, vocab, nrows=10, ncols=100, color_nums=None)

shape of t_normed is (500, 100)


solar,0.402,-0.247,0.686,-0.055,0.056,-0.259,0.02,-0.087,0.146,-0.032,0.083,-0.078,0.066,-0.023,0.116,-0.024,0.04,-0.092,0.022,-0.024,0.013,-0.004,0.017,-0.065,0.026,0.026,-0.036,0.044,-0.054,0.075,-0.051,-0.046,0.027,0.157,-0.084,-0.012,-0.054,0.105,-0.091,-0.013,0.025,-0.02,0.088,-0.019,0.041,-0.023,0.041,-0.037,0.039,-0.025,0.031,-0.013,0.016,-0.054,0.13,-0.09,0.037,-0.034,-0.012,-0.066,0.042,-0.022,0.036,0.031,-0.058,0.016,0.035,0.006,-0.04,-0.002,-0.018,0.051,0.029,0.001,0.035,-0.021,-0.045,0.011,0.002,0.015,-0.054,-0.015,0.002,0.038,0.022,-0.029,-0.011,-0.024,-0.037,0.021,0.015,-0.007,0.001,-0.002,0.014,-0.001,0.016,-0.031,0.025,-0.013
climate,0.248,-0.124,-0.291,-0.595,0.228,-0.219,0.165,-0.056,-0.196,0.186,-0.246,-0.043,-0.054,-0.04,0.009,0.022,-0.106,-0.032,0.038,-0.026,0.172,0.24,-0.004,-0.017,-0.134,-0.188,-0.081,0.022,-0.041,-0.006,0.091,0.009,-0.005,0.003,-0.03,-0.062,-0.002,-0.061,-0.037,0.008,0.004,0.063,0.076,0.013,0.03,0.049,0.013,-0.024,0.012,0.035,0.042,0.049,-0.01,-0.016,0.014,0.002,-0.02,0.028,-0.018,0.01,0.035,0.021,0.008,0.012,-0.026,0.012,-0.005,-0.008,-0.035,0.018,-0.028,0.011,0.023,0.009,0.007,0.002,0.023,0.025,0.011,-0.006,-0.021,0.01,-0.008,-0.002,0.022,0.012,-0.003,-0.002,0.01,0.017,0.016,0.015,-0.007,-0.004,0.017,-0.005,-0.013,0.008,-0.001,-0.018
surface,0.258,-0.105,-0.052,0.344,-0.029,-0.033,0.031,0.081,-0.652,-0.232,-0.219,0.237,0.267,0.171,-0.013,-0.109,-0.053,0.033,-0.071,0.052,0.035,0.006,-0.017,-0.194,0.05,-0.03,-0.029,0.042,0.059,0.059,-0.082,0.027,-0.002,0.047,-0.011,0.001,-0.06,0.01,0.007,0.019,0.028,0.003,0.042,-0.032,-0.023,0.001,0.049,-0.047,0.016,0.021,0.011,-0.007,0.005,-0.021,-0.017,-0.021,-0.015,-0.001,0.023,0.044,0.013,0.005,-0.009,-0.005,0.021,0.02,-0.006,0.012,0.009,-0.003,-0.004,0.02,0.008,-0.02,0.003,0.003,-0.012,-0.013,0.004,0.018,0.005,0.008,0.016,0.006,0.012,-0.0,0.001,-0.001,-0.013,-0.03,0.011,-0.009,0.001,-0.0,-0.007,0.004,0.007,-0.002,-0.001,-0.003
energy,0.214,-0.145,0.066,0.107,0.216,0.495,0.342,0.583,0.096,0.152,0.046,-0.015,-0.113,-0.094,-0.044,0.09,-0.073,-0.001,-0.019,0.007,0.012,0.017,-0.1,-0.106,-0.003,-0.136,0.054,-0.023,-0.065,0.032,0.023,0.084,-0.101,0.002,-0.074,0.053,0.03,0.034,0.062,0.015,0.037,0.019,0.007,0.043,-0.007,0.005,0.039,0.024,0.028,-0.007,0.017,0.027,0.024,-0.014,-0.013,0.009,0.042,-0.009,0.019,-0.009,-0.029,-0.007,-0.003,-0.004,-0.003,-0.005,0.023,0.03,-0.01,-0.005,-0.013,-0.014,-0.005,0.013,0.001,-0.007,-0.005,0.006,-0.012,0.006,0.011,-0.008,0.001,-0.005,-0.011,-0.009,0.012,0.008,-0.003,0.005,0.014,0.001,0.007,-0.007,-0.007,0.0,0.012,-0.007,0.006,0.002
temperature,0.228,-0.07,-0.288,0.105,-0.131,-0.257,-0.119,0.184,-0.017,-0.387,0.535,-0.24,-0.259,-0.146,-0.099,0.035,-0.089,-0.05,-0.061,-0.082,0.162,0.112,-0.028,-0.057,-0.041,-0.032,-0.082,0.017,0.073,-0.011,-0.038,-0.023,-0.011,-0.016,-0.026,0.0,-0.058,0.022,0.049,0.022,-0.001,-0.015,0.061,0.064,-0.025,0.016,-0.023,0.026,-0.012,-0.012,-0.016,0.046,-0.048,-0.059,-0.023,0.026,0.011,-0.02,-0.002,0.018,0.008,0.037,0.04,0.01,-0.017,0.036,-0.009,0.021,0.009,0.017,0.037,-0.003,-0.022,0.005,0.026,0.007,0.0,0.019,0.004,-0.01,-0.004,-0.001,-0.015,-0.013,0.019,0.016,0.005,0.027,-0.006,0.042,0.025,0.019,0.011,-0.012,0.021,0.023,-0.014,-0.015,0.014,-0.02
atmosphere,0.221,-0.158,-0.077,0.23,0.098,0.291,-0.118,-0.499,-0.067,0.087,-0.131,-0.192,-0.271,-0.161,-0.339,0.255,-0.108,0.076,0.227,0.043,-0.04,-0.177,0.05,-0.081,-0.12,0.053,-0.062,0.02,0.02,0.061,-0.042,-0.022,0.003,0.055,0.014,-0.007,-0.053,-0.009,-0.046,-0.023,0.008,0.009,0.024,0.006,0.031,-0.002,0.034,0.008,0.036,-0.009,0.002,-0.003,0.005,-0.023,-0.0,-0.013,0.02,-0.025,-0.022,-0.001,0.003,0.0,-0.009,0.004,0.007,0.002,0.01,0.016,-0.017,0.002,0.027,-0.001,0.021,0.011,-0.012,0.007,-0.024,-0.004,-0.011,0.009,0.017,-0.022,0.027,0.008,-0.01,0.006,-0.024,0.005,-0.014,-0.017,0.01,0.007,-0.005,-0.001,0.007,0.027,0.028,-0.026,0.0,0.003
year,0.148,0.141,-0.007,-0.151,-0.2,0.169,-0.196,0.102,0.235,-0.321,-0.348,-0.1,0.351,-0.459,-0.185,-0.223,-0.108,-0.01,0.1

The SVD produces three matrices. We can think of the first one "T" as having rows corresponding to our vocabulary. And we can take as many columns as we want.

In [14]:
def get_word_vector(w, vocab, mat):
    return norm_vec(mat[vocab.index(w)])

def compare_word_vectors(w1, w2, vocab, mat):
    return np.dot(get_word_vector(w1, vocab, mat), get_word_vector(w2, vocab, mat))

def get_doc_vector(doc, vocab, td_mat):
    s = np.zeros(td_mat.shape[1])
    for w in doc:
        if w in vocab:
            s = s + get_word_vector(w, vocab, td_mat)
    return s

In [15]:
compare_word_vectors("close", "closer", vocab, T_reduced)

0.35740699296626

In [16]:
def most_similar(w, vocab, mat, n=10):
    sims = []
    for w2 in vocab:
        if w2 == w:
            continue
        sims.append([w2, compare_word_vectors(w, w2, vocab, mat), fdist[w2]])
    return sorted(sims, key=lambda item: item[1], reverse=True)[:n]

In [17]:
most_similar("hemisphere", vocab, T_reduced)

[['southern', 0.4488782260403123, 294],
 ['towards', 0.33313226661084394, 91],
 ['receives', 0.3235520518768023, 67],
 ['left', 0.29159382508994247, 61],
 ['northern', 0.28488731336725465, 552],
 ['shorter', 0.23934271265987003, 57],
 ['moving', 0.20902264316028146, 75],
 ['opposite', 0.1996276302353479, 84],
 ['start', 0.1840723807729869, 54],
 ['tilted', 0.18384494623394226, 241]]

In [18]:
most_similar("tilt", vocab, T_reduced)

[['degree', 0.3228569803519825, 78],
 ['reason', 0.30407654384690863, 66],
 ['towards', 0.27750722120792914, 91],
 ['varies', 0.2666088408787079, 103],
 ['planetary', 0.23758245195019978, 80],
 ['moving', 0.18485086806387577, 75],
 ['measure', 0.1617084708496929, 59],
 ['distribution', 0.14805305804147975, 90],
 ['summers', 0.1404002811104602, 54],
 ['uranus', 0.1359018810979812, 53]]

## PCA for plotting

I'll show you a way of squishing any higher dimensional matrix down to a smaller number of dimensions for the purpose of making plots.

In [19]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def squish_matrix(X, ncomponents=2):
    pca = PCA(n_components=ncomponents)
    return pca.fit_transform(X)

def double_squish(X, ncomponents=2):
    pca = PCA(n_components=50)
    reduc = pca.fit_transform(X)
    return TSNE(n_components=ncomponents, random_state=0, perplexity=15).fit_transform(reduc)

def alt_squish(X, ncomponents=2):
    return TSNE(n_components=ncomponents, random_state=0, perplexity=15).fit_transform(X)

In [20]:
%matplotlib widget
def plot_matrix(mat, labels, number_to_plot=10, figsize=(10, 10), c="red"):
    if mat.shape[1] > 3:
        print("too many dimensions")
        return
    if number_to_plot > mat.shape[0]:
        number_to_plot = mat.shape[0]
    fig=plt.figure(figsize=figsize, dpi= 80, facecolor='w', edgecolor='k')
    if mat.shape[1] == 3:
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(mat[:, 0][:number_to_plot], mat[:, 1][:number_to_plot], mat[:, 2][:number_to_plot], c=c)
        for i in range(number_to_plot):
            ax.text(mat[i, 0], mat[i, 1], mat[i, 2], labels[i])

    else:
        plt.scatter(mat[:, 0][:number_to_plot], mat[:, 1][:number_to_plot], c=c)
        for i in range(number_to_plot):
            plt.annotate(labels[i], mat[i])

ModuleNotFoundError: No module named 'ipympl'

In [ ]:
sm = squish_matrix(T_normed, 3)
plot_matrix(sm, vocab, 10, figsize=(10, 10))

In [ ]:
sm = squish_matrix(T_normed, 3)
plot_matrix(sm, vocab, 10, figsize=(10, 10))

Let's instead plot a few selected words

In [ ]:
selected_words = ["tilt", "hemisphere", "northern", "southern", "side", "day", "night", "moon", "spin", "closer", "close", "near", "far", "farther"]
folded_vocab_list = []
found_vocab = []
for w in selected_words:
    if w not in vocab:
        continue
    found_vocab.append(w)
    v = get_word_vector(w, vocab, T_normed)
    # v = sm[vocab.index(w)]
    folded_vocab_list.append(v)
folded_vocab_matrix = np.array(folded_vocab_list)

In [ ]:
show_labeled_table(folded_vocab_matrix, None, found_vocab, nrows=25, color_nums=None)

In [ ]:
sm = squish_matrix(folded_vocab_matrix, 2)
plot_matrix(sm, found_vocab, 10, figsize=(5, 5))

In [ ]:
import copy, random
def plot_similar(the_word, vocab, mat, dims=2, n=10):
    similar_words = [w[0] for w in most_similar(the_word, vocab, mat)]
    folded_vocab_list = [mat[vocab.index(the_word)]]
    found_vocab = [the_word]
    colors = ["red"]
    for w in similar_words[:n]:
        if w not in vocab:
            continue
        found_vocab.append(w)
        v = mat[vocab.index(w)]
        folded_vocab_list.append(v)
        colors.append("blue")
    new_vocab = copy.deepcopy(vocab)
    random.shuffle(new_vocab)
    for w in new_vocab[:n]:
        if w not in vocab:
            continue
        found_vocab.append(w)
        v = mat[vocab.index(w)]
        folded_vocab_list.append(v)
        colors.append("green")
    folded_vocab_matrix = np.array(folded_vocab_list)
    squished_matrix = alt_squish(folded_vocab_matrix, dims)
    plot_matrix(squished_matrix, found_vocab, 50, c=colors)

In [ ]:
%matplotlib notebook
plot_similar("hemisphere", vocab, T_normed, 2)